In [1]:
using Distributed
const nprocs=4
addprocs(nprocs)
r=[]
for p in workers()
    push!(r,RemoteChannel(()->Channel{Any}(1), p))
end



@everywhere using Gadfly
@everywhere using Plots
@everywhere using RegionTrees
@everywhere using StaticArrays: SVector
@everywhere using GeometricalPredicates
@everywhere using Colors
@everywhere include("VoronoiDelaunay.jl")
@everywhere include("Package.jl")

@everywhere const shape=Package.shape
@everywhere const gamma=Package.gamma
@everywhere const Courant=Package.Courant
@everywhere const thetaC=Package.thetaC
@everywhere const kappa=Package.kappa
@everywhere const nu = Package.nu
@everywhere const khi=Package.khi
@everywhere const scale_fact=Package.scale_fact

@everywhere const tf=Package.tf
@everywhere const Dt=Package.Dt
@everywhere const mtilda=Package.mtilda
@everywhere const Vtilda=Package.Vtilda

@everywhere const TOL=Package.TOL




@everywhere const scale_down=Package.scale_down
@everywhere const width1 = Package.width1
@everywhere const min_coord2 = Package.min_coord2
@everywhere const max_coord2 = Package.max_coord2
@everywhere const marge_min=Package.marge_min
@everywhere const marge_max=Package.marge_max

In [22]:
#For the steps see Documentation page 24

n=45*45
IV=[[1,0.,0.,0.] for i in 1:n]


GP=[Point2D(min_coord2+((i-1)÷45+0.5)*width1/45,min_coord2+((i-1)%45+0.5)*width1/45) for i in 1:n]



CV=[Package.from_Intensive_to_Conservative(IV[i]) for i in 1:n]

CV[23*45+23][4]=1.
IV[23*45+23]=Package.from_Conservative_to_Intensive(CV[23*45+23])
x,y=getx(GP[23*45+23]),gety(GP[23*45+23])
GP[22*45+22]=Point2D(x-width1/(45*sqrt(2)),y-width1/(45*sqrt(2)))
GP[24*45+22]=Point2D(x+width1/(45*sqrt(2)),y-width1/(45*sqrt(2)))
GP[22*45+24]=Point2D(x-width1/(45*sqrt(2)),y+width1/(45*sqrt(2)))
GP[24*45+24]=Point2D(x+width1/(45*sqrt(2)),y+width1/(45*sqrt(2)))

sum=0.
for i in 1:n
    sum+=1/(CV[i][1]/mtilda+1/Vtilda)
end
Ktilda = (scale_down*scale_fact)^2/sum

r=[]
for p in workers()
    push!(r,RemoteChannel(()->Channel{Any}(1), p))
end


#I/ Initialize the mesh and decompose domain
mesh=Package.Mesh(GP,CV,IV)
Package.decomposition_periodicbc(mesh,nprocs)
#Package.decomposition_symmetricbc(mesh,nprocs)

#II/ First parallel loop
@sync begin
    for i in 1:nprocs
        @async r[i]=remotecall(Package.parallel_initial_one,i+1,mesh,i+1)
    end
end

#III/ Update all data depending on the geometry for ghost cells
Package.ghost_IV_init(r)

#IV/ Build the tree for timestep calculation
Tree4time=RegionTrees.Cell(SVector(1.0, 1.0), SVector(1-eps(Float64),1-eps(Float64)),[Point2D(0.,0.),[0.,0.],1])    
for i in 1:nprocs
    Package.treebuilding4timestep(Tree4time,fetch(r[i]))
end

#IV/ Compute in parallel the timesteps
@sync begin
    for i in 1:nprocs
        @async r[i]=remotecall(Package.computedeltat,i+1,fetch(r[i]),Tree4time,r)
    end
end

#V/ Find the first time δt at which there will be active cells
currentK=Package.ghost_time_init(r)

#VI/ Enter the second parallel computation 
@sync begin
    for i in 1:nprocs
        @async r[i]=remotecall(Package.parallel_initial_two,i+1,fetch(r[i]),Ktilda)
    end
end



In [23]:
t=0
whichhalf=Int64[]
deltat=0
drifting=50
set_default_plot_size(20cm, 20cm)
p=[]
affichage = 1.

#VII/ Enter the computation loop
while t<tf
    
    #1 - Build the tree for timestep calculation
    Tree4time=RegionTrees.Cell(SVector(1.0, 1.0), SVector(1-eps(Float64),1-eps(Float64)),[Point2D(0.,0.),[0.,0.],1])    
    for i in 1:nprocs
        Package.treebuilding4timestep(Tree4time,fetch(r[i]))
    end

    #1 - Compute in parallel the timesteps for active cells
    @sync begin
        for i in 1:nprocs
            @async r[i]=remotecall(Package.deltat_update,i+1,fetch(r[i]),Tree4time,r,currentK)
        end
    end
    
    #2 - Update the gradient and the speed of ghost cells
    Package.ghost_grad_speed(r)
    
    #3 - Compute fluxes for active faces
    @sync begin
        for i in 1:nprocs
            @async r[i]=remotecall(Package.parallel_boucle_one,i+1,fetch(r[i]),currentK)
        end
    end
    

    mesh=Package.mesh_construction(r)
    if currentK <= 6
        push!(p,Package.getplot(mesh))
    end
    
    #4 - Compute the next time where particles are active
    deltat,currentK,whichhalf=Package.time_update(mesh,whichhalf,currentK)
    t+=deltat
    
    #5 - Shift all mesh generating points
    Package.drift(mesh,deltat)
    Ktilda=Package.computeKtilda(mesh)

    #6 - Decompose your domain
    Package.decomposition_periodicbc(mesh,nprocs)
    #Package.decomposition_symmetricbc(mesh,nprocs)
    
    #7 - Enter a parallel computation 
    @sync begin
        for i in 1:nprocs
            @async r[i]=remotecall(Package.parallel_boucle_two,i+1,mesh,i+1,currentK)
        end
    end
    
    
    #8 - Update intensive variables of ghost cells as well as data depen- ding on the geometry
    Package.ghost_IV(r)
    
    #9 - Enter a parallel computation
    @sync begin
        for i in 1:nprocs
            @async r[i]=remotecall(Package.parallel_boucle_three,i+1,fetch(r[i]),currentK,Ktilda)
        end
    end
    if rand()<affichage
        println(t)
        affichage=affichage*0.67
    end

end

0.001953125
0.00390625
0.015625
0.0234375
0.0390625
0.0625
0.0703125
0.30078125
0.5234375
0.6171875


RemoteException: On worker 5:
DomainError with -0.0013338946237326408:
sqrt will only return a complex result if called with a complex argument. Try sqrt(Complex(x)).
throw_complex_domainerror at ./math.jl:31
sqrt at ./math.jl:493
update_IV at /Users/ezra/Desktop/Master's Thesis/Package.jl:1626
parallel_boucle_two at /Users/ezra/Desktop/Master's Thesis/Package.jl:2325
#109 at /Users/sabae/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.2/Distributed/src/process_messages.jl:288
run_work_thunk at /Users/sabae/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.2/Distributed/src/process_messages.jl:79
run_work_thunk at /Users/sabae/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.2/Distributed/src/process_messages.jl:88
#102 at ./task.jl:268